# Sprint 2 : 

In [293]:
####Importation des packagins :

import pandas as pd
import numpy as np

#Packaging de visualisation
import seaborn as sns
import matplotlib.pyplot as plt

#packaging preprocessing
from sklearn.preprocessing import OrdinalEncoder

#selection de model
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

#packaging model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

#packagng metrics
from sklearn.metrics import make_scorer, mean_squared_error



import warnings
warnings.filterwarnings('ignore')

In [294]:
df = pd.read_csv('laptop_cleaned.csv')

In [295]:
X,y = df.drop('Price', axis = 1), df['Price']

# 1. KNeighborsRegressor

In [296]:
#instanciation des parametres 
knn = KNeighborsRegressor()
cross = KFold(n_splits=5, shuffle=True, random_state=42)

In [312]:
model_init = cross_val_score(estimator=knn, 
                X=X,y=y, 
                cv=cross,  
                scoring=make_scorer(mean_squared_error))

# 2. Mesure du score 

In [313]:
#### definition d'une fonction calculant le rmse : 
def rmse(score):
    moy = score.mean()
    rmse = np.sqrt(moy)
    return round(rmse,2)

In [299]:
rmse(model_init)

167.47

# 3. Amelioration du modèle :

- On va tenter d'améliorer :
    - le nombre de split optimal sur notre KFold
    - n_neighbors, metric


In [300]:
parametre_grid = {"n_neighbors":np.arange(2,30), "metric": ["euclidean", "manhattan"] }

In [301]:
grid = GridSearchCV(estimator=KNeighborsRegressor(), 
                    param_grid=parametre_grid, 
                    cv=cross)

In [302]:
grid.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsRegressor(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [303]:
best_grid = grid.best_estimator_
best_grid

KNeighborsRegressor(metric='manhattan', n_neighbors=3)

In [304]:
best_model_grid = cross_val_score(estimator= best_grid, 
                X=X,y=y, 
                cv=cross,  
                scoring=make_scorer(mean_squared_error))

In [305]:
dict_score_split = {}
for i in range(2,20):
    cross = KFold(n_splits=i, shuffle=True, random_state=42)
    best_model_grid = cross_val_score(estimator= best_grid, 
                X=X,y=y, 
                cv=cross,  
                scoring=make_scorer(mean_squared_error))
    dict_score_split[i] = float(rmse(best_model_grid))
    print("split ",i, ": rmse =", dict_score_split[i], "$")

split  2 : rmse = 162.1 $
split  3 : rmse = 160.18 $
split  4 : rmse = 157.6 $
split  5 : rmse = 154.71 $
split  6 : rmse = 155.34 $
split  7 : rmse = 153.54 $
split  8 : rmse = 154.3 $
split  9 : rmse = 154.9 $
split  10 : rmse = 155.01 $
split  11 : rmse = 154.1 $
split  12 : rmse = 154.47 $
split  13 : rmse = 154.79 $
split  14 : rmse = 153.21 $
split  15 : rmse = 152.5 $
split  16 : rmse = 152.89 $
split  17 : rmse = 151.49 $
split  18 : rmse = 153.28 $
split  19 : rmse = 152.93 $


In [306]:
best_n_split = min(dict_score_split, key = dict_score_split.get)

In [307]:
#### Meilleur modèle KNN ### 

In [308]:
best_cross = KFold(n_splits=best_n_split, shuffle = True, random_state=42)

In [309]:
best_model = cross_val_score(estimator=best_grid, X = X, y = y, cv = best_cross, scoring=make_scorer(mean_squared_error))

In [310]:
print("Prédiction du prix avec l'amélioration du modèle à ", rmse(best_model), "$ près") 

Prédiction du prix avec l'amélioration du modèle à  151.49 $ près


Amélioration de la prédiction du prix de : 15.98 $
